In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Sickle_Cell_Anemia"
cohort = "GSE46471"

# Input paths
in_trait_dir = "../../input/GEO/Sickle_Cell_Anemia"
in_cohort_dir = "../../input/GEO/Sickle_Cell_Anemia/GSE46471"

# Output paths
out_data_file = "../../output/preprocess/Sickle_Cell_Anemia/GSE46471.csv"
out_gene_data_file = "../../output/preprocess/Sickle_Cell_Anemia/gene_data/GSE46471.csv"
out_clinical_data_file = "../../output/preprocess/Sickle_Cell_Anemia/clinical_data/GSE46471.csv"
json_path = "../../output/preprocess/Sickle_Cell_Anemia/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Mechanisms of haptoglobin protection against hemoglobin peroxidation triggered endothelial damage"
!Series_summary	"Extracellular hemoglobin (Hb) has been recognized as a disease trigger in hemolytic conditions such as sickle cell disease, malaria and blood transfusion. In vivo, many of the adverse effects of free Hb can be attenuated by the Hb scavenger acute phase protein haptoglobin (Hp). The primary physiologic disturbances that can be caused be free Hb are found within the cardiovascular system and Hb triggered oxidative toxicity towards the endothelium has been promoted as a potential mechanism. The molecular mechanisms of this toxicity as well as of the protective activities of Hp are not yet clear. Within this study we systematically investigated the structural, biochemical and cell biologic nature of Hb toxicity in an endothelial cell system under peroxidative stress."
!Series_summary	"We identified two principal mechanisms of oxidative H

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import os
import pandas as pd
import json
from typing import Optional, Callable, Dict, Any
import numpy as np

# 1. Gene Expression Data Availability
# From the background information, this dataset is about protein and cellular processes,
# likely containing gene expression data
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability

# For trait (Sickle Cell Anemia):
# The dataset doesn't explicitly show which samples have sickle cell anemia
# The study is about haptoglobin protection against hemoglobin peroxidation, which is related to sickle cell anemia
# But we don't see a clear variable for it in the sample characteristics
trait_row = None

# For age:
# No age information is provided in the sample characteristics
age_row = None

# For gender:
# No gender information is provided in the sample characteristics
gender_row = None

# 2.2 Data Type Conversion
def convert_trait(value):
    """Convert trait data to binary (0/1)"""
    if value is None:
        return None
    
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # This is a placeholder since trait_row is None in this dataset
    return None

def convert_age(value):
    """Convert age data to continuous numeric values"""
    if value is None:
        return None
    
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # This is a placeholder since age_row is None in this dataset
    return None

def convert_gender(value):
    """Convert gender data to binary (0=female, 1=male)"""
    if value is None:
        return None
    
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # This is a placeholder since gender_row is None in this dataset
    return None

# 3. Save Metadata
# Determine trait data availability
is_trait_available = trait_row is not None

# Save initial filtering metadata
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Since trait_row is None, we skip this substep


False

### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. First, let's examine the structure of the matrix file to understand its format
import gzip

# Peek at the first few lines of the file to understand its structure
with gzip.open(matrix_file, 'rt') as file:
    # Read first 100 lines to find the header structure
    for i, line in enumerate(file):
        if '!series_matrix_table_begin' in line:
            print(f"Found data marker at line {i}")
            # Read the next line which should be the header
            header_line = next(file)
            print(f"Header line: {header_line.strip()}")
            # And the first data line
            first_data_line = next(file)
            print(f"First data line: {first_data_line.strip()}")
            break
        if i > 100:  # Limit search to first 100 lines
            print("Matrix table marker not found in first 100 lines")
            break

# 3. Now try to get the genetic data with better error handling
try:
    gene_data = get_genetic_data(matrix_file)
    print(gene_data.index[:20])
except KeyError as e:
    print(f"KeyError: {e}")
    
    # Alternative approach: manually extract the data
    print("\nTrying alternative approach to read the gene data:")
    with gzip.open(matrix_file, 'rt') as file:
        # Find the start of the data
        for line in file:
            if '!series_matrix_table_begin' in line:
                break
                
        # Read the headers and data
        import pandas as pd
        df = pd.read_csv(file, sep='\t', index_col=0)
        print(f"Column names: {df.columns[:5]}")
        print(f"First 20 row IDs: {df.index[:20]}")
        gene_data = df


Found data marker at line 67
Header line: "ID_REF"	"GSM1130756"	"GSM1130757"	"GSM1130758"	"GSM1130759"	"GSM1130760"	"GSM1130761"	"GSM1130762"	"GSM1130763"	"GSM1130764"	"GSM1130765"	"GSM1130766"	"GSM1130767"	"GSM1130768"	"GSM1130769"	"GSM1130770"	"GSM1130771"	"GSM1130772"	"GSM1130773"	"GSM1130774"	"GSM1130775"	"GSM1130776"	"GSM1130777"	"GSM1130778"	"GSM1130779"	"GSM1130780"	"GSM1130781"	"GSM1130782"	"GSM1130783"	"GSM1130784"	"GSM1130785"	"GSM1130786"	"GSM1130787"	"GSM1130788"	"GSM1130789"	"GSM1130790"	"GSM1130791"	"GSM1130792"	"GSM1130793"	"GSM1130794"	"GSM1130795"	"GSM1130796"	"GSM1130797"	"GSM1130798"	"GSM1130799"	"GSM1130800"
First data line: 1	-3.04E-02	-7.44E-02	2.51E-02	4.59E-02	1.32E-02	-3.86E-01	1.63E-03	1.35E-01	2.90E-02	-2.74E-01	4.99E-02	1.31E-01	1.85E-01	5.61E-02	-1.69E-02	-1.39E-01	-1.33E-01	1.85E-01	2.19E-01	2.12E-01	2.51E-01	3.07E-01	1.25E-01	2.12E-02	7.70E-02	3.12E-01	1.26E-03	-6.42E-02	3.38E-02	8.79E-02	2.62E-01	-6.44E-02	7.47E-03	-3.78E-02	1.43E-01	2.37E-02	-1.54E-01	1

### Step 4: Gene Identifier Review

In [5]:
# The identifiers in the gene expression data are numeric IDs (1, 2, 3, etc.), which are likely
# probe identifiers from a microarray. These are not human gene symbols and need to be mapped.
# Numeric IDs like this typically correspond to positions on a microarray chip and need to be
# mapped to actual gene symbols using platform annotation files.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Let's first examine the structure of the SOFT file before trying to parse it
import gzip

# Look at the first few lines of the SOFT file to understand its structure
print("Examining SOFT file structure:")
try:
    with gzip.open(soft_file, 'rt') as file:
        # Read first 20 lines to understand the file structure
        for i, line in enumerate(file):
            if i < 20:
                print(f"Line {i}: {line.strip()}")
            else:
                break
except Exception as e:
    print(f"Error reading SOFT file: {e}")

# 2. Now let's try a more robust approach to extract the gene annotation
# Instead of using the library function which failed, we'll implement a custom approach
try:
    # First, look for the platform section which contains gene annotation
    platform_data = []
    with gzip.open(soft_file, 'rt') as file:
        in_platform_section = False
        for line in file:
            if line.startswith('^PLATFORM'):
                in_platform_section = True
                continue
            if in_platform_section and line.startswith('!platform_table_begin'):
                # Next line should be the header
                header = next(file).strip()
                platform_data.append(header)
                # Read until the end of the platform table
                for table_line in file:
                    if table_line.startswith('!platform_table_end'):
                        break
                    platform_data.append(table_line.strip())
                break
    
    # If we found platform data, convert it to a DataFrame
    if platform_data:
        import pandas as pd
        import io
        platform_text = '\n'.join(platform_data)
        gene_annotation = pd.read_csv(io.StringIO(platform_text), delimiter='\t', 
                                      low_memory=False, on_bad_lines='skip')
        print("\nGene annotation preview:")
        print(preview_df(gene_annotation))
    else:
        print("Could not find platform table in SOFT file")
        
        # Try an alternative approach - extract mapping from other sections
        with gzip.open(soft_file, 'rt') as file:
            for line in file:
                if 'ANNOTATION information' in line or 'annotation information' in line:
                    print(f"Found annotation information: {line.strip()}")
                if line.startswith('!Platform_title') or line.startswith('!platform_title'):
                    print(f"Platform title: {line.strip()}")
            
except Exception as e:
    print(f"Error processing gene annotation: {e}")


Examining SOFT file structure:


Line 0: ^DATABASE = GeoMiame
Line 1: !Database_name = Gene Expression Omnibus (GEO)
Line 2: !Database_institute = NCBI NLM NIH
Line 3: !Database_web_link = http://www.ncbi.nlm.nih.gov/geo
Line 4: !Database_email = geo@ncbi.nlm.nih.gov
Line 5: ^SERIES = GSE46471
Line 6: !Series_title = Mechanisms of haptoglobin protection against hemoglobin peroxidation triggered endothelial damage
Line 7: !Series_geo_accession = GSE46471
Line 8: !Series_status = Public on Apr 30 2015
Line 9: !Series_submission_date = Apr 29 2013
Line 10: !Series_last_update_date = Feb 22 2018
Line 11: !Series_pubmed_id = 23995229
Line 12: !Series_summary = Extracellular hemoglobin (Hb) has been recognized as a disease trigger in hemolytic conditions such as sickle cell disease, malaria and blood transfusion. In vivo, many of the adverse effects of free Hb can be attenuated by the Hb scavenger acute phase protein haptoglobin (Hp). The primary physiologic disturbances that can be caused be free Hb are found within the ca


Gene annotation preview:
{'ID': [1, 2, 3, 4, 5], 'COL': [266, 266, 266, 266, 266], 'ROW': [170, 168, 166, 164, 162], 'NAME': ['GE_BrightCorner', 'DarkCorner', 'DarkCorner', 'DarkCorner', 'DarkCorner'], 'SPOT_ID': ['GE_BrightCorner', 'DarkCorner', 'DarkCorner', 'DarkCorner', 'DarkCorner'], 'CONTROL_TYPE': ['pos', 'pos', 'pos', 'pos', 'pos'], 'REFSEQ': [nan, nan, nan, nan, nan], 'GB_ACC': [nan, nan, nan, nan, nan], 'GENE': [nan, nan, nan, nan, nan], 'GENE_SYMBOL': [nan, nan, nan, nan, nan], 'GENE_NAME': [nan, nan, nan, nan, nan], 'UNIGENE_ID': [nan, nan, nan, nan, nan], 'ENSEMBL_ID': [nan, nan, nan, nan, nan], 'TIGR_ID': [nan, nan, nan, nan, nan], 'ACCESSION_STRING': [nan, nan, nan, nan, nan], 'CHROMOSOMAL_LOCATION': [nan, nan, nan, nan, nan], 'CYTOBAND': [nan, nan, nan, nan, nan], 'DESCRIPTION': [nan, nan, nan, nan, nan], 'GO_ID': [nan, nan, nan, nan, nan], 'SEQUENCE': [nan, nan, nan, nan, nan], 'SPOT_ID.1': [nan, nan, nan, nan, nan], 'ORDER': [1, 2, 3, 4, 5]}


### Step 6: Gene Identifier Mapping

In [7]:
# 1. Observe the gene identifiers and annotation data
# From previous analysis:
# - Gene expression data has numeric IDs (1, 2, 3, etc.)
# - Gene annotation data has columns for ID and GENE_SYMBOL

# 2. Extract the columns for mapping
# The ID field in the gene annotation corresponds to the gene identifiers in expression data
# The GENE_SYMBOL field contains the gene symbols we want to map to
mapping_df = gene_annotation[['ID', 'GENE_SYMBOL']]

# Convert ID to string to match the format in gene_data
mapping_df['ID'] = mapping_df['ID'].astype(str)

# Rename 'GENE_SYMBOL' to 'Gene' as required by apply_gene_mapping function
mapping_df = mapping_df.rename(columns={'GENE_SYMBOL': 'Gene'})

# 3. Apply the gene mapping to convert probe-level data to gene expression data
# We'll use the library function that handles the many-to-many mapping correctly
gene_data = apply_gene_mapping(gene_data, mapping_df)

# Preview the gene data to verify the mapping worked
print("Gene expression data after mapping:")
print(f"Shape: {gene_data.shape}")
print(f"First few gene symbols: {list(gene_data.index[:10])}")

# Save the gene expression data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to: {out_gene_data_file}")

# Update the metadata to mark the dataset as not usable
validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available,
    is_biased=True,  # We don't have trait data, so mark as biased
    df=pd.DataFrame(),  # Empty dataframe since we can't proceed without trait data
    note="Dataset contains gene expression data but lacks trait information for Sickle Cell Anemia"
)


Gene expression data after mapping:
Shape: (18379, 45)
First few gene symbols: ['A1BG', 'A1CF', 'A2BP1', 'A2LD1', 'A2M', 'A2ML1', 'A3GALT2', 'A4GALT', 'A4GNT', 'AAAS']


/tmp/ipykernel_13897/1685547521.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mapping_df['ID'] = mapping_df['ID'].astype(str)


Gene expression data saved to: ../../output/preprocess/Sickle_Cell_Anemia/gene_data/GSE46471.csv
Abnormality detected in the cohort: GSE46471. Preprocessing failed.


False

### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the obtained gene expression data
print(f"Gene data shape before normalization: {gene_data.shape}")

# Normalize gene symbols using NCBI Gene database
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {normalized_gene_data.shape}")

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Check if trait data is available before proceeding with clinical data extraction
if trait_row is None:
    print("Trait row is None. Cannot extract trait information from clinical data.")
    # Create an empty dataframe for clinical features
    clinical_features = pd.DataFrame()
    
    # Create an empty dataframe for linked data
    linked_data = pd.DataFrame()
    
    # Validate and save cohort info
    validate_and_save_cohort_info(
        is_final=True, 
        cohort=cohort, 
        info_path=json_path, 
        is_gene_available=True, 
        is_trait_available=False,  # Trait data is not available
        is_biased=True,  # Not applicable but required
        df=pd.DataFrame(),  # Empty dataframe
        note=f"Dataset contains gene expression data but lacks clear trait indicators for {trait} status."
    )
    print("Data was determined to be unusable due to missing trait indicators and was not saved")
else:
    try:
        # Get the file paths for the matrix file to extract clinical data
        _, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
        
        # Get raw clinical data from the matrix file
        _, clinical_raw = get_background_and_clinical_data(matrix_file)
        
        # Verify clinical data structure
        print("Raw clinical data shape:", clinical_raw.shape)
        
        # Extract clinical features using the defined conversion functions
        clinical_features = geo_select_clinical_features(
            clinical_df=clinical_raw,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        
        print("Clinical features:")
        print(clinical_features)
        
        # Save clinical features to file
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        clinical_features.to_csv(out_clinical_data_file)
        print(f"Clinical features saved to {out_clinical_data_file}")
        
        # 3. Link clinical and genetic data
        linked_data = geo_link_clinical_genetic_data(clinical_features, normalized_gene_data)
        print(f"Linked data shape: {linked_data.shape}")
        print("Linked data preview (first 5 rows, first 5 columns):")
        print(linked_data.iloc[:5, :5])
        
        # 4. Handle missing values
        print("Missing values before handling:")
        print(f"  Trait ({trait}) missing: {linked_data[trait].isna().sum()} out of {len(linked_data)}")
        if 'Age' in linked_data.columns:
            print(f"  Age missing: {linked_data['Age'].isna().sum()} out of {len(linked_data)}")
        if 'Gender' in linked_data.columns:
            print(f"  Gender missing: {linked_data['Gender'].isna().sum()} out of {len(linked_data)}")
        
        gene_cols = [col for col in linked_data.columns if col not in [trait, 'Age', 'Gender']]
        print(f"  Genes with >20% missing: {sum(linked_data[gene_cols].isna().mean() > 0.2)}")
        print(f"  Samples with >5% missing genes: {sum(linked_data[gene_cols].isna().mean(axis=1) > 0.05)}")
        
        cleaned_data = handle_missing_values(linked_data, trait)
        print(f"Data shape after handling missing values: {cleaned_data.shape}")
        
        # 5. Evaluate bias in trait and demographic features
        is_trait_biased = False
        if len(cleaned_data) > 0:
            trait_biased, cleaned_data = judge_and_remove_biased_features(cleaned_data, trait)
            is_trait_biased = trait_biased
        else:
            print("No data remains after handling missing values.")
            is_trait_biased = True
        
        # 6. Final validation and save
        is_usable = validate_and_save_cohort_info(
            is_final=True, 
            cohort=cohort, 
            info_path=json_path, 
            is_gene_available=True, 
            is_trait_available=True, 
            is_biased=is_trait_biased, 
            df=cleaned_data,
            note=f"Dataset contains gene expression data for {trait} analysis."
        )
        
        # 7. Save if usable
        if is_usable and len(cleaned_data) > 0:
            os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
            cleaned_data.to_csv(out_data_file)
            print(f"Linked data saved to {out_data_file}")
        else:
            print("Data was determined to be unusable or empty and was not saved")
            
    except Exception as e:
        print(f"Error processing data: {e}")
        # Handle the error case by still recording cohort info
        validate_and_save_cohort_info(
            is_final=True, 
            cohort=cohort, 
            info_path=json_path, 
            is_gene_available=True, 
            is_trait_available=False,  # Mark as not available due to processing issues
            is_biased=True, 
            df=pd.DataFrame(),  # Empty dataframe
            note=f"Error processing data for {trait}: {str(e)}"
        )
        print("Data was determined to be unusable and was not saved")

Gene data shape before normalization: (18379, 45)
Gene data shape after normalization: (17901, 45)


Normalized gene data saved to ../../output/preprocess/Sickle_Cell_Anemia/gene_data/GSE46471.csv
Trait row is None. Cannot extract trait information from clinical data.
Abnormality detected in the cohort: GSE46471. Preprocessing failed.
Data was determined to be unusable due to missing trait indicators and was not saved
